In [1]:
import pydicom as pydcm
import matplotlib.pyplot as plt
import pandas as pd
import sys
from pydicom.datadict import dictionary_VR
from pydicom import dcmread,Dataset
from pydicom.filewriter import dcmwrite
import numpy as np
#import cv2
import random
import time
import datetime


from google.cloud import storage
from io import BytesIO
client = storage.Client()

In [2]:
def blob_to_image(bucket, blob_name):
    blob = bucket.get_blob(blob_name)
    blobstring = blob.download_as_string()
    dicomfile = BytesIO(blobstring)
    dicomread = dcmread(dicomfile)
    #plt.imshow(dicomread.pixel_array)
    return dicomread

def img_diff(raw, deid):
    diff = raw.pixel_array-deid.pixel_array
    result = np.count_nonzero(diff)
    return(result)    

def find(list,thing): 
    "finds indices of something in a list"
    return [i for i, j in enumerate(list) if j == thing]


def blob_to_df(filelist, bucket):
    SOP = []
    patient = []
    study = []
    series = []
    DICOM = []
    for i in range(len(filelist)):
        dicom = blob_to_image(bucket, filelist[i])
        DICOM.append(dicom)
        patient.append(dicom.PatientID)
        study.append(dicom.StudyInstanceUID)
        series.append(dicom.SeriesInstanceUID)
        SOP.append(dicom.SOPInstanceUID)
    
    d = {'dicom': DICOM, 'patientID': patient, 'studyID': study, 'seriesID': series, 
         'SOPID': SOP}
    df = pd.DataFrame(d)
    return(df)


In [ ]:
bucket_in = client.get_bucket('midi-import')

blobs_in = bucket_in.list_blobs(prefix='ds1_1-23')
filelist_in = [item.name for item in blobs_in] #extracting file names from blobs

df = blob_to_df(filelist_in[1:], bucket_in)
#df_in = blob_to_df(filelist_in[1:], bucket_in)

In [4]:
import re
def filterLength(n):
    if len(n)>=6:
        return True
    else:
        return False


In [34]:
#automatic code final

file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []
newvalue = []

#loops through each dicom file imported
for i in range(len(df)):
    file = df.iloc[i][0]
    #loops through each element in the dicom file
    for elem in file.iterall():
        #only looks at elements of vr 'LO', 'SH', 'LT', 'ST', 'UT','IS'
        if elem.VR in vrs:
            #ignores elements that don't contain any numeric characters
            if str(elem.value).isalpha() == False:
                str1 = str(elem.value)
                try:
                    #creates list of all continuous numeric strings of characters in the element
                    x = re.findall('[0-9]+', str1)
                    #narrows this list to only strings of length 6 or longer
                    x2 = list(filter(filterLength, x))
                    if len(x2) != 0:
                        #appends the element info to lists. This is not necessary for de-id, but is used to double check the results
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(str(elem.value))
                        #loops through the list of continuous numeric strings in the element
                        for n in x2:
                            if len(n) == 8:
                                #if the string's length is 8, check if it is a valid yyyymmdd date
                                yr = n[:4]
                                m = n[4:6]
                                d = n[6:]
                                if 1895<=int(yr)<=2023 and 0<int(m)<=12 and 0<int(d)<=31 and elem.VR != 'IS':
                                    #if it is a valid yyyymmdd date and isn't in VR IS, reformat it to yyyy-mm-dd
                                    str3 = yr+'-'+m+'-'+d
                                    str4 = str1.replace(n,str3)
                                    elem.value = str4
                                else:
                                    #replaces the 8 length int string with nothing
                                    str2 = str1.replace(n,'')
                                    elem.value = str2
                            else:
                                #replaces the int string (not 8 length) with nothing
                                str2 = str1.replace(n,'')
                                elem.value = str2
                        newvalue.append(str(elem.value))
                                
                except:
                    print(elem)
                                      
                        
    

In [ ]:
#uploads changed files to gcp bucket
bucket = client.get_bucket('midi-import')
for i in range(1,len(filelist_in)):
    filepath = 'preprocessed/ds1_1-23preprocessed6-27/'+filelist_in[i]
    blob = bucket.blob(filepath)
    df.iloc[i-1][0].save_as('test6.dcm')
    blob.upload_from_filename('test6.dcm')
    